# **Домашнее задание 3**

**1. Постройте нейронную сеть(берем простую линейную сеть, которую разбирали на уроке: меняем число слоев, число нейронов , типы активации, тип оптимизатора)  на датасет from sklearn.datasets import load_boston.** 






In [19]:
%tensorflow_version 2.x

In [39]:

import pandas as pd
import numpy as np

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error as mae, mean_squared_error as mse, r2_score as r2

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [7]:
boston = load_boston()

df = pd.DataFrame(boston.data, columns=boston.feature_names)
df.head(3)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03


In [8]:
data = boston.data
target = boston.target

In [10]:
train_data, test_data, train_target, test_target = train_test_split(data, target, test_size=0.20)

In [13]:
train_data.shape

(404, 13)

In [32]:
model = keras.Sequential([
  keras.layers.Dense(64, activation='relu',
                      input_shape=(train_data.shape[1],)),
  keras.layers.Dense(32, activation='relu'),
  keras.layers.Dense(1)
])

In [33]:
model.compile(loss='mse',
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['mae'])

In [34]:
model.fit(
    train_data,
    train_target,
    epochs=5,
    verbose = 0
)

In [35]:
model.evaluate(train_data, train_target)

13/13 [==============================] - 0s 1ms/step - loss: 63.6750 - mae: 6.3390


[63.67496871948242, 6.338968276977539]

**2. Измените функцию потерь и метрику для этой задачи. Постройте 10-15 вариантов и сведите результаты их работы в таблицу  Опишите, какого результата вы добились от нейросети? Что помогло вам улучшить ее точность?**

In [57]:
results = []

for n in range(16):
  for metrics in ['mean_absolute_error', 'mean_squared_error']:
      for loss in ['mse', 'mae']:
        model = keras.Sequential([
        keras.layers.Dense(64, activation='relu',input_shape=(train_data.shape[1],)),
        keras.layers.Dense(32, activation='relu'),
        keras.layers.Dense(1)
        ])

        result = {}

        model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
                      loss=loss,
                      metrics=metrics)
            
        model.fit(train_data, train_target,
                  epochs=200, verbose = 0)
        
        predictions = model.predict(test_data)

        result['n'] = n
        result['metrics'] = metrics
        result['loss'] = loss
        result['r2'] = round(r2(test_target, predictions.copy().flatten()),5)
        result['rmse'] = round(np.sqrt(mse(test_target, predictions.copy().flatten())),5)
        result['mse'] = round(mse(test_target, predictions.copy().flatten()),5)
        result['mae'] = round(mae(test_target, predictions.copy().flatten()),5)


        results.append(result)


result = pd.DataFrame(results)
result


,n,metrics,loss,r2,rmse,mse,mae
0,0,mean_absolute_error,mse,0.65879,5.26216,27.69032,3.57748
1,0,mean_absolute_error,mae,0.58159,5.82711,33.95522,3.74742
2,0,mean_squared_error,mse,0.58165,5.82672,33.95063,4.29803
3,0,mean_squared_error,mae,0.64062,5.40043,29.16463,3.50115
4,1,mean_absolute_error,mse,0.62093,5.54645,30.76313,3.92810
...,...,...,...,...,...,...,...
59,14,mean_squared_error,mae,0.62329,5.52914,30.57138,3.51661
60,15,mean_absolute_error,mse,0.68138,5.08501,25.85734,3.48369
61,15,mean_absolute_error,mae,0.69553,4.97081,24.70898,3.48416
62,15,mean_squared_error,mse,0.69400,4.98326,24.83285,3.54146


In [62]:
result.loc[(result['metrics']== 'mean_absolute_error') & (result['loss'] == 'mse')].mean()

n        7.500000
r2       0.680626
rmse     5.070312
mse     25.918351
mae      3.608514
dtype: float64

In [63]:
result.loc[(result['metrics']== 'mean_absolute_error') & (result['loss'] == 'mae')].mean()

n        7.500000
r2       0.629511
rmse     5.466466
mse     30.066462
mae      3.748294
dtype: float64

In [64]:
result.loc[(result['metrics']== 'mean_squared_error') & (result['loss'] == 'mse')].mean()

n        7.500000
r2       0.630756
rmse     5.416844
mse     29.965421
mae      3.946987
dtype: float64

In [65]:
result.loc[(result['metrics']== 'mean_squared_error') & (result['loss'] == 'mae')].mean()

n        7.500000
r2       0.624960
rmse     5.502696
mse     30.435840
mae      3.669066
dtype: float64

Модель лучше всего показала себя при выборе в качестве  метрики mae и функции потерь mse. Увеличение эпох до 200 значительно улучшило результат.